# NLP Pipeline with spaCy

[spaCy](https://spacy.io/) is a widely used python library with a comprehensive feature set for fast text processing in multiple languages. 

The usage of the tokenization and annotation engines requires the installation of language models. The features we will use in this chapter only require the small models, the larger models also include word vectors that we will cover in chapter 15.

![spaCy](assets/spacy.jpg)

## Setup

### Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
import sys
from pathlib import Path

import pandas as pd

import spacy
from spacy import displacy
from textacy.extract import ngrams, entities

### SpaCy Language Model Installation

In addition to the `spaCy` library, we need [language models](https://spacy.io/usage/models).

#### English

Only need to run once.

In [3]:
%%bash
python -m spacy download en_core_web_sm

# more comprehensive models:
# {sys.executable} -m spacy download en_core_web_md
# {sys.executable} -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


#### Spanish

[Spanish language models](https://spacy.io/models/es#es_core_news_sm) trained on [AnCora Corpus](http://clic.ub.edu/corpus/) and [WikiNER](http://schwa.org/projects/resources/wiki/Wikiner)

Only need to run once.

In [4]:
%%bash
python -m spacy download es_core_news_sm

# more comprehensive model:
# {sys.executable} -m spacy download es_core_news_md

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


Create shortcut names

In [5]:
%%bash
python -m spacy link en_core_web_sm en --force;
python -m spacy link es_core_news_sm es --force;

✔ Linking successful
/home/stefan/.pyenv/versions/miniconda3-latest/envs/ml4t/lib/python3.7/site-packages/en_core_web_sm
-->
/home/stefan/.pyenv/versions/miniconda3-latest/envs/ml4t/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Linking successful
/home/stefan/.pyenv/versions/miniconda3-latest/envs/ml4t/lib/python3.7/site-packages/es_core_news_sm
-->
/home/stefan/.pyenv/versions/miniconda3-latest/envs/ml4t/lib/python3.7/site-packages/spacy/data/es
You can now load the model via spacy.load('es')


#### Validate Installation

In [6]:
# validate installation
!{sys.executable} -m spacy validate

✔ Loaded compatibility table

====================== Installed models (spaCy v2.3.0) ======================
ℹ spaCy installation:
/home/stefan/.pyenv/versions/miniconda3-latest/envs/ml4t/lib/python3.7/site-packages/spacy

TYPE      NAME              MODEL             VERSION                            
package   es-core-news-sm   es_core_news_sm   2.3.0   ✔
package   en-core-web-sm    en_core_web_sm    2.3.0   ✔
link      en                en_core_web_sm    2.3.0   ✔
link      es                es_core_news_sm   2.3.0   ✔



## Get Data

In [7]:
DATA_DIR = Path('..', 'data')

- [BBC Articles](http://mlg.ucd.ie/datasets/bbc.html), use raw text files ([download](http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip))
    - Data already included in [data](../data) directory, just unzip before first-time use.
- [TED2013](http://opus.nlpl.eu/TED2013.php), a parallel corpus of TED talk subtitles in 15 langugages (sample provided) in `results/TED` subfolder of this directory.

## SpaCy Pipeline & Architecture

### The Processing Pipeline

When you call a spaCy model on a text, spaCy 

1) tokenizes the text to produce a `Doc` object. 

2) passes the `Doc` object through the processing pipeline that may be customized, and for the default models consists of
- a tagger, 
- a parser and 
- an entity recognizer. 

Each pipeline component returns the processed Doc, which is then passed on to the next component.

![Architecture](assets/pipeline.svg)

### Key Data Structures

The central data structures in spaCy are the **Doc** and the **Vocab**. Text annotations are also designed to allow a single source of truth:

- The **`Doc`** object owns the sequence of tokens and all their annotations. `Span` and `Token` are views that point into it. It is constructed by the `Tokenizer`, and then modified in place by the components of the pipeline. 
- The **`Vocab`** object owns a set of look-up tables that make common information available across documents. 
- The **`Language`** object coordinates these components. It takes raw text and sends it through the pipeline, returning an annotated document. It also orchestrates training and serialization.

![Architecture](assets/spaCy-architecture.svg)

## SpaCy in Action

### Create & Explore the Language Object

Once installed and linked, we can instantiate a spaCy language model and then call it on a document. As a result, spaCy produces a Doc object that tokenizes the text and processes it according to configurable pipeline components that by default consist of a tagger, a parser, and a named-entity recognizer.

In [8]:
nlp = spacy.load('en') 

In [9]:
type(nlp)

spacy.lang.en.English

In [10]:
nlp.lang

'en'

In [11]:
spacy.info('en')


=========================== Info about model 'en' ===========================

lang             en                            
name             core_web_sm                   
license          MIT                           
author           Explosion                     
url              https://explosion.ai          
email            contact@explosion.ai          
description      English multi-task CNN trained on OntoNotes. Assigns context-specific token vectors, POS tags, dependency parse and named entities.
sources          [{'name': 'OntoNotes 5', 'url': 'https://catalog.ldc.upenn.edu/LDC2013T19', 'license': 'commercial (licensed by Explosion)'}]
pipeline         ['tagger', 'parser', 'ner']   
version          2.3.0                         
spacy_version    >=2.3.0,<2.4.0                
parent_package   spacy                         
labels           {'tagger': ['$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'ADD', 'AFX', 'CC', 'CD', 'DT', 'EX', 'FW', 'HYPH', 'IN', 'JJ', 'JJR', 'JJS

{'lang': 'en',
 'name': 'core_web_sm',
 'license': 'MIT',
 'author': 'Explosion',
 'url': 'https://explosion.ai',
 'email': 'contact@explosion.ai',
 'description': 'English multi-task CNN trained on OntoNotes. Assigns context-specific token vectors, POS tags, dependency parse and named entities.',
 'sources': [{'name': 'OntoNotes 5',
   'url': 'https://catalog.ldc.upenn.edu/LDC2013T19',
   'license': 'commercial (licensed by Explosion)'}],
 'pipeline': ['tagger', 'parser', 'ner'],
 'version': '2.3.0',
 'spacy_version': '>=2.3.0,<2.4.0',
 'parent_package': 'spacy',
 'accuracy': {'las': 89.6760148148,
  'uas': 91.5566789084,
  'token_acc': 99.7576498212,
  'las_per_type': {'compound': {'p': 90.4808743169,
    'r': 92.2143016262,
    'f': 91.339364519},
   'root': {'p': 88.3934259141, 'r': 91.5122337268, 'f': 89.9257963125},
   'advcl': {'p': 68.2166446499, 'r': 65.0214051876, 'f': 66.5807117071},
   'prep': {'p': 85.668083186, 'r': 86.4286527298, 'f': 86.0466873159},
   'det': {'p': 97.5

In [12]:
def get_attributes(f):
    print([a for a in dir(f) if not a.startswith('_')], end=' ')

In [13]:
get_attributes(nlp)

['Defaults', 'add_pipe', 'begin_training', 'create_pipe', 'disable_pipes', 'entity', 'evaluate', 'factories', 'from_bytes', 'from_disk', 'get_pipe', 'has_pipe', 'lang', 'linker', 'make_doc', 'matcher', 'max_length', 'meta', 'parser', 'path', 'pipe', 'pipe_factories', 'pipe_labels', 'pipe_names', 'pipeline', 'preprocess_gold', 'rehearse', 'remove_pipe', 'rename_pipe', 'replace_pipe', 'resume_training', 'tagger', 'tensorizer', 'to_bytes', 'to_disk', 'tokenizer', 'update', 'use_params', 'vocab'] 

### Explore the Pipeline

Let’s illustrate the pipeline using a simple sentence:

In [14]:
sample_text = 'Apple is looking at buying U.K. startup for $1 billion'
doc = nlp(sample_text)

In [15]:
get_attributes(doc)

['cats', 'char_span', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_disk', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'is_nered', 'is_parsed', 'is_sentenced', 'is_tagged', 'lang', 'lang_', 'mem', 'merge', 'noun_chunks', 'noun_chunks_iterator', 'print_tree', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_extension', 'similarity', 'tensor', 'text', 'text_with_ws', 'to_array', 'to_bytes', 'to_disk', 'to_json', 'to_utf8_array', 'user_data', 'user_hooks', 'user_span_hooks', 'user_token_hooks', 'vector', 'vector_norm', 'vocab'] 

In [16]:
doc.is_parsed

True

In [17]:
doc.is_sentenced

True

In [18]:
doc.is_tagged

True

In [19]:
doc.text

'Apple is looking at buying U.K. startup for $1 billion'

In [20]:
get_attributes(doc.vocab)

['add_flag', 'cfg', 'data_dir', 'from_bytes', 'from_disk', 'get_vector', 'has_vector', 'lang', 'length', 'lex_attr_getters', 'load_extra_lookups', 'lookups', 'lookups_extra', 'morphology', 'prune_vectors', 'reset_vectors', 'set_vector', 'strings', 'to_bytes', 'to_disk', 'vectors', 'vectors_length', 'writing_system'] 

In [21]:
doc.vocab.length

487

#### Explore `Token` annotations

The parsed document content is iterable and each element has numerous attributes produced by the processing pipeline. The below sample illustrates how to access the following attributes:

In [22]:
pd.Series([token.text for token in doc])

0       Apple
1          is
2     looking
3          at
4      buying
5        U.K.
6     startup
7         for
8           $
9           1
10    billion
dtype: object

In [23]:
pd.DataFrame([[t.text, t.lemma_, t.pos_, t.tag_, t.dep_, t.shape_, t.is_alpha, t.is_stop]
              for t in doc],
             columns=['text', 'lemma', 'pos', 'tag', 'dep', 'shape', 'is_alpha', 'is_stop'])

,text,lemma,pos,tag,dep,shape,is_alpha,is_stop
0,Apple,Apple,PROPN,NNP,nsubj,Xxxxx,True,False
1,is,be,AUX,VBZ,aux,xx,True,True
2,looking,look,VERB,VBG,ROOT,xxxx,True,False
3,at,at,ADP,IN,prep,xx,True,True
4,buying,buy,VERB,VBG,pcomp,xxxx,True,False
5,U.K.,U.K.,PROPN,NNP,compound,X.X.,False,False
6,startup,startup,NOUN,NN,dobj,xxxx,True,False
7,for,for,ADP,IN,prep,xxx,True,True
8,$,$,SYM,$,quantmod,$,False,False
9,1,1,NUM,CD,compound,d,False,False


#### Visualize POS Dependencies

We can visualize the syntactic dependency in a browser or notebook

In [24]:
options = {'compact': True, 'bg': 'white',
           'color': 'black', 'font': 'Source Sans Pro', 'notebook': True}

In [25]:
displacy.render(doc, style='dep', options=options)

#### Visualize Named Entities

In [26]:
displacy.render(doc, style='ent', jupyter=True)

### Read BBC Data

We will now read a larger set of 2,225 BBC News articles (see GitHub for data source details) that belong to five categories and are stored in individual text files. We 
- call the .glob() method of the pathlib’s Path object, 
- iterate over the resulting list of paths, 
- read all lines of the news article excluding the heading in the first line, and 
- append the cleaned result to a list

In [27]:
files = (DATA_DIR / 'bbc').glob('**/*.txt')
bbc_articles = []
for i, file in enumerate(sorted(list(files))):
    with file.open(encoding='latin1') as f:
        lines = f.readlines()
        body = ' '.join([l.strip() for l in lines[1:]]).strip()
        bbc_articles.append(body)

In [28]:
len(bbc_articles)

2225

In [29]:
bbc_articles[0]

'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL\'s existing customers for high-

### Parse first article through Pipeline

In [30]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [31]:
doc = nlp(bbc_articles[0])
type(doc)

spacy.tokens.doc.Doc

### Detect sentence boundary
Sentence boundaries are calculated from the syntactic parse tree, so features such as punctuation and capitalisation play an important but non-decisive role in determining the sentence boundaries. 

Usually this means that the sentence boundaries will at least coincide with clause boundaries, even given poorly punctuated text.

spaCy computes sentence boundaries from the syntactic parse tree so that punctuation and capitalization play an important but not decisive role. As a result, boundaries will coincide with clause boundaries, even for poorly punctuated text.

We can access the parsed sentences using the .sents attribute:

In [32]:
sentences = [s for s in doc.sents]
sentences[:3]

[Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.  ,
 The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales.,
 TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.]

In [33]:
get_attributes(sentences[0])

['as_doc', 'char_span', 'conjuncts', 'doc', 'end', 'end_char', 'ent_id', 'ent_id_', 'ents', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'kb_id', 'kb_id_', 'label', 'label_', 'lefts', 'lemma_', 'lower_', 'merge', 'n_lefts', 'n_rights', 'noun_chunks', 'orth_', 'remove_extension', 'rights', 'root', 'sent', 'sentiment', 'set_extension', 'similarity', 'start', 'start_char', 'string', 'subtree', 'tensor', 'text', 'text_with_ws', 'to_array', 'upper_', 'vector', 'vector_norm', 'vocab'] 

In [34]:
pd.DataFrame([[t.text, t.pos_, spacy.explain(t.pos_)] for t in sentences[0]], 
             columns=['Token', 'POS Tag', 'Meaning']).head(15)

,Token,POS Tag,Meaning
0,Quarterly,ADJ,adjective
1,profits,NOUN,noun
2,at,ADP,adposition
3,US,PROPN,proper noun
4,media,NOUN,noun
5,giant,NOUN,noun
6,TimeWarner,PROPN,proper noun
7,jumped,VERB,verb
8,76,NUM,numeral
9,%,NOUN,noun


In [35]:
options = {'compact': True, 'bg': '#09a3d5',
           'color': 'white', 'font': 'Source Sans Pro'}
displacy.render(sentences[0].as_doc(), style='dep', jupyter=True, options=options)

In [36]:
for t in sentences[0]:
    if t.ent_type_:
        print('{} | {} | {}'.format(t.text, t.ent_type_, spacy.explain(t.ent_type_)))

Quarterly | DATE | Absolute or relative dates or periods
US | GPE | Countries, cities, states
TimeWarner | ORG | Companies, agencies, institutions, etc.
76 | PERCENT | Percentage, including "%"
% | PERCENT | Percentage, including "%"
1.13bn | MONEY | Monetary values, including unit
the | DATE | Absolute or relative dates or periods
three | DATE | Absolute or relative dates or periods
months | DATE | Absolute or relative dates or periods
to | DATE | Absolute or relative dates or periods
December | DATE | Absolute or relative dates or periods
639 | MONEY | Monetary values, including unit
year | DATE | Absolute or relative dates or periods
- | DATE | Absolute or relative dates or periods
earlier | DATE | Absolute or relative dates or periods


In [37]:
displacy.render(sentences[0].as_doc(), style='ent', jupyter=True)

### Named Entity-Recognition with textacy

spaCy enables named entity recognition using the .ent_type_ attribute:

Textacy makes access to the named entities that appear in the first article easy:

In [38]:
entities = [e.text for e in entities(doc)]
pd.Series(entities).value_counts().head()

TimeWarner        7
AOL               5
fourth quarter    3
8%                2
AOL Europe        2
dtype: int64

### N-Grams with textacy

N-grams combine N consecutive tokens. This can be useful for the bag-of-words model because, depending on the textual context, treating, e.g, ‘data scientist’ as a single token may be more meaningful than the two distinct tokens ‘data’ and ‘scientist’.

Textacy makes it easy to view the ngrams of a given length n occurring with at least min_freq times:

In [39]:
pd.Series([n.text for n in ngrams(doc, n=2, min_freq=2)]).value_counts()

fourth quarter     3
Time Warner        2
quarter profits    2
AOL Europe         2
company said       2
dtype: int64

### The spaCy streaming Pipeline API

To pass a larger number of documents through the processing pipeline, we can use spaCy’s streaming API as follows:

In [40]:
iter_texts = (bbc_articles[i] for i in range(len(bbc_articles)))
for i, doc in enumerate(nlp.pipe(iter_texts, batch_size=50, n_threads=8)):
    if i % 100 == 0:
        print(i, end = ' ')
    assert doc.is_parsed

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 

### Multi-language Features

spaCy includes trained language models for English, German, Spanish, Portuguese, French, Italian and Dutch, as well as a multi-language model for named-entity recognition. Cross-language usage is straightforward since the API does not change.

We will illustrate the Spanish language model using a parallel corpus of TED talk subtitles. For this purpose, we instantiate both language models

#### Create a Spanish Language Object

In [41]:
model = {}
for language in ['en', 'es']:
    model[language] = spacy.load(language) 

#### Read bilingual TED2013 samples

In [42]:
text = {}
path = Path('data', 'TED')
for language in ['en', 'es']:
    file_name = path /  'TED2013_sample.{}'.format(language)
    text[language] = file_name.read_text()

#### Sentence Boundaries English vs Spanish

In [43]:
parsed, sentences = {}, {}
for language in ['en', 'es']:
    parsed[language] = model[language](text[language])
    sentences[language] = list(parsed[language].sents)
    print('Sentences:', language, len(sentences[language]))

Sentences: en 21
Sentences: es 22


In [44]:
for i, (en, es) in enumerate(zip(sentences['en'], sentences['es']), 1):
    print('\n', i)
    print('English:\t', en)
    print('Spanish:\t', es)
    if i > 5: 
        break


 1
English:	 There's a tight and surprising link between the ocean's health and ours, says marine biologist Stephen Palumbi.
Spanish:	 Existe una estrecha y sorprendente relación entre nuestra salud y la salud del océano, dice el biologo marino Stephen Palumbi.

 2
English:	 He shows how toxins at the bottom of the ocean food chain find their way into our bodies, with a shocking story of toxic contamination from a Japanese fish market.
Spanish:	 Nos muestra, através de una impactante historia acerca de la contaminación tóxica en el mercado pesquero japonés, como las toxinas de la cadena alimenticia del fondo oceánico llegan a nuestro cuerpo.

 3
English:	 His work points a way forward for saving the oceans' health -- and humanity's.
Spanish:	 fish,health,mission blue,oceans,science 899 Stephen Palumbi: Siguiendo el camino del mercurio.

 4
English:	 fish,health,mission blue,oceans,science 899 Stephen Palumbi:
Spanish:	 El océano puede ser una cosa muy complicada.

 5
English:	 Followi

#### POS Tagging English vs Spanish

In [45]:
pos = {}
for language in ['en', 'es']:
    pos[language] = pd.DataFrame([[t.text, t.pos_, spacy.explain(t.pos_)] for t in sentences[language][0]],
                                 columns=['Token', 'POS Tag', 'Meaning'])

In [46]:
bilingual_parsed = pd.concat([pos['en'], pos['es']], axis=1)
bilingual_parsed.head(15)

,Token,POS Tag,Meaning,Token,POS Tag,Meaning
0,There,PRON,pronoun,Existe,VERB,verb
1,'s,AUX,auxiliary,una,DET,determiner
2,a,DET,determiner,estrecha,ADJ,adjective
3,tight,ADJ,adjective,y,CCONJ,coordinating conjunction
4,and,CCONJ,coordinating conjunction,sorprendente,ADJ,adjective
5,surprising,ADJ,adjective,relación,NOUN,noun
6,link,NOUN,noun,entre,ADP,adposition
7,between,ADP,adposition,nuestra,DET,determiner
8,the,DET,determiner,salud,NOUN,noun
9,ocean,NOUN,noun,y,CCONJ,coordinating conjunction


In [47]:
displacy.render(sentences['es'][0].as_doc(), style='dep', jupyter=True, options=options)